In [1]:
from pathlib import Path
import pandas as pd
import sqlite3

def main():
    # === 1. Setup paths ===
    try:
        project_dir = Path(__file__).parent.resolve()  # Automatically gets the script's directory
    except NameError:
        project_dir = Path().resolve()
    data_folder = project_dir.parent / "zhvi_raw_files"
    sqlite_path = project_dir.parent / "data" / "zhvi_data.db"

    if sqlite_path.exists():
        print("🧹 Removing existing database...")
        sqlite_path.unlink()
    file_list = list(data_folder.glob("*.csv"))  # Grabs all CSV files

    if not file_list:
        raise FileNotFoundError(f"No CSV files found in {data_folder}")

    # === 2. Load & process all CSV files ===
    all_dfs = []

    for file in file_list:
        # Extract 18th character of filename that represents bedroom count corresponding to the dataset (e.g., ‘Zip_zhvi_bdrmcnt_3_uc.csv’ = 3)
        base_name = file.name
        bedroom_digit = base_name[17]  # Position 18
        bedroom_count = "5+" if bedroom_digit == "5" else bedroom_digit

        df = pd.read_csv(file)
        df["FileName"] = base_name
        df["BedroomCount"] = bedroom_count
        all_dfs.append(df)

    # === 3. Concatenate all data ===
    df_all = pd.concat(all_dfs, ignore_index=True)

    # === 4. Filter to RegionType == "zip" to remove any duplicate headers amongst the combined files ===
    df_all = df_all[df_all["RegionType"] == "zip"].copy()

    # === 5. Drop 'FileName' column ===
    df_all.drop(columns=["FileName"], inplace=True)

    # === 6. Unpivot wide date columns into rows ===
    id_vars = [
        "RegionID", "BedroomCount", "SizeRank", "RegionName", "RegionType",
        "StateName", "State", "City", "Metro", "CountyName"
    ]
    value_vars = [col for col in df_all.columns if col not in id_vars]
    df_long = pd.melt(
        df_all,
        id_vars=id_vars,
        value_vars=value_vars,
        var_name="Date",
        value_name="HomeValue"
    )

    # === 7. Data type conversions ===
    df_long["HomeValue"] = pd.to_numeric(df_long["HomeValue"], errors="coerce")
    df_long["Date"] = pd.to_datetime(df_long["Date"], errors="coerce")
    df_long["BedroomCount"] = df_long["BedroomCount"].astype(str)
    df_long["RegionName"] = df_long["RegionName"].astype(str)

    # === 8. Reorder columns to match original ===
    column_order = [
        "RegionID", "BedroomCount", "SizeRank", "RegionName", "RegionType",
        "StateName", "State", "City", "Metro", "CountyName", "Date", "HomeValue"
    ]
    df_final = df_long[column_order]
    print(f"Final dataset contains {df_final.shape[0]:,} rows and {df_final.shape[1]} columns.")


    # === 9. Save to SQLite ===
    conn = sqlite3.connect(sqlite_path)
    df_final.to_sql("zhvi_data", conn, if_exists="replace", index=False)
    print(f"Saving {df_final.shape[0]:,} rows to SQLite")
    conn.commit()
    conn.close()

    # === 10. Export cleaned data for Tableau ===
    tableau_export_path = project_dir.parent / "data" / "zhvi_tableau_ready.csv"
    df_final.to_csv(tableau_export_path, index=False)
    print(f"✅ Exported cleaned data to {tableau_export_path}")


if __name__ == "__main__":
    main()

🧹 Removing existing database...
Final dataset contains 301,340 rows and 12 columns.
Saving 301,340 rows to SQLite
✅ Exported cleaned data to C:\Users\ericm\OneDrive\Documents\GitHub_Sync\zhvi-TN-housing-trends\data\zhvi_tableau_ready.csv
